In [1]:
# Epochs ve batch size'ı ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd

In [2]:
df44 = pd.read_csv("diabet.csv",header=0,infer_datetime_format=True)
df44.shape

(768, 9)

In [3]:
df44.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df44 = df44.iloc[:,0:9]
df44.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df44.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [6]:
dataset=numpy.array(df44)
dataset

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

# 1- Toplu İş Boyutu ve Dönem Sayısı Nasıl Ayarlanır# 

In [8]:
# KerasClassifier için gerekli olan modeli oluşturma 
def create_model():
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# model oluşturma
model = KerasClassifier(build_fn=create_model, verbose=0)
# Izgara arama parametrelerini tanımladık.
batch_size = [2, 4, 8, 16, 32, 64,128]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.735677 using {'batch_size': 8, 'epochs': 100}
0.621094 (0.011500) with: {'batch_size': 2, 'epochs': 10}
0.707031 (0.003189) with: {'batch_size': 2, 'epochs': 50}
0.713542 (0.034104) with: {'batch_size': 2, 'epochs': 100}
0.626302 (0.016367) with: {'batch_size': 4, 'epochs': 10}
0.690104 (0.036966) with: {'batch_size': 4, 'epochs': 50}
0.731771 (0.021710) with: {'batch_size': 4, 'epochs': 100}
0.652344 (0.014616) with: {'batch_size': 8, 'epochs': 10}
0.677083 (0.008027) with: {'batch_size': 8, 'epochs': 50}
0.735677 (0.032106) with: {'batch_size': 8, 'epochs': 100}
0.566406 (0.019918) with: {'batch_size': 16, 'epochs': 10}
0.677083 (0.020256) with: {'batch_size': 16, 'epochs': 50}
0.700521 (0.025976) with: {'batch_size': 16, 'epochs': 100}
0.519531 (0.013902) with: {'batch_size': 32, 'epochs': 10}
0.634115 (0.049855) with: {'batch_size': 32, 'epochs': 50}
0.678385 (0.001841) with: {'batch_size': 32, 'epochs': 100}
0.548177 (0.094201) with: {'batch_size': 64, 'epochs': 10}
0.6575

# 2- Eğitim Optimizasyon Algoritmasını Ayarlama

In [13]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [14]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(optimizer='adam'):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.686198 using {'optimizer': 'Adam'}
0.656250 (0.030758) with: {'optimizer': 'SGD'}
0.683594 (0.009568) with: {'optimizer': 'RMSprop'}
0.591146 (0.024150) with: {'optimizer': 'Adagrad'}
0.632812 (0.052698) with: {'optimizer': 'Adadelta'}
0.686198 (0.008027) with: {'optimizer': 'Adam'}
0.664062 (0.028705) with: {'optimizer': 'Adamax'}
0.686198 (0.012890) with: {'optimizer': 'Nadam'}


# 3- Öğrenme Hızını ve İvmeyi Ayarlama

In [15]:
# Öğrenme hızını ve momentumu ızgarada aramak için scikit-learn'i kullanalım
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD

In [16]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(learn_rate=0.01, momentum=0):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Modeli derleme
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
learn_rate = [0.001, 0.005, 0.0015, 0.01, 0.025]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.694010 using {'learn_rate': 0.001, 'momentum': 0.6}
0.665365 (0.016053) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.667969 (0.025315) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.690104 (0.036690) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.694010 (0.021236) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.665365 (0.026557) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.647135 (0.027126) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.670573 (0.032106) with: {'learn_rate': 0.005, 'momentum': 0.0}
0.670573 (0.036690) with: {'learn_rate': 0.005, 'momentum': 0.2}
0.644531 (0.011049) with: {'learn_rate': 0.005, 'momentum': 0.4}
0.664062 (0.031412) with: {'learn_rate': 0.005, 'momentum': 0.6}
0.649740 (0.026557) with: {'learn_rate': 0.005, 'momentum': 0.8}
0.649740 (0.026557) with: {'learn_rate': 0.005, 'momentum': 0.9}
0.662760 (0.018688) with: {'learn_rate': 0.0015, 'momentum': 0.0}
0.654948 (0.036966) with: {'learn_rate': 0.0015, 'momentum': 0.2}
0.662760 (0.035277) with: {'

# 4- Ağ Ağırlığı Başlatma Nasıl Ayarlanır

In [17]:
# Ağırlık başlatmayı ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [18]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(init_mode='uniform'):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer=init_mode, activation='relu'))
	model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.730469 using {'init_mode': 'uniform'}
0.730469 (0.016573) with: {'init_mode': 'uniform'}
0.695312 (0.013902) with: {'init_mode': 'lecun_uniform'}
0.727865 (0.012075) with: {'init_mode': 'normal'}
0.651042 (0.024774) with: {'init_mode': 'zero'}
0.712240 (0.016053) with: {'init_mode': 'glorot_normal'}
0.662760 (0.031466) with: {'init_mode': 'glorot_uniform'}
0.703125 (0.013902) with: {'init_mode': 'he_normal'}
0.669271 (0.016367) with: {'init_mode': 'he_uniform'}


# 5- Nöron Aktivasyon Fonksiyonu Nasıl Ayarlanır

In [19]:
# Aktivasyon işlevini ızgarada aramak için scikit-learn'i kullanın
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [20]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(activation='relu'):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation=activation))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.727865 using {'activation': 'softplus'}
0.654948 (0.013279) with: {'activation': 'softmax'}
0.727865 (0.026557) with: {'activation': 'softplus'}
0.665365 (0.024774) with: {'activation': 'softsign'}
0.722656 (0.019401) with: {'activation': 'relu'}
0.667969 (0.005524) with: {'activation': 'tanh'}
0.670573 (0.038051) with: {'activation': 'sigmoid'}
0.688802 (0.016053) with: {'activation': 'hard_sigmoid'}
0.722656 (0.025315) with: {'activation': 'linear'}


# 6- Bırakma Düzenlemesini Ayarlama

In [21]:
# Bırakma oranını ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

In [22]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(dropout_rate=0.0, weight_constraint=0):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(weight_constraint)))
	model.add(Dropout(dropout_rate))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
#Izgara arama parametrelerini tanımladık.
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.723958 using {'dropout_rate': 0.3, 'weight_constraint': 1}
0.712240 (0.027866) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.716146 (0.026748) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.707031 (0.003189) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.699219 (0.009568) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.692708 (0.019488) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.722656 (0.022326) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.703125 (0.016573) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.722656 (0.019918) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.712240 (0.006639) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.704427 (0.006639) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.708333 (0.015733) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.708333 (0.009207) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.713542 (0.006639) with: {'dropout_rate': 0.2, 'weight_constraint': 

# 7- Gizli Katmandaki Nöron Sayısı Nasıl Ayarlanır

In [23]:
# Nöron sayısını ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

In [24]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(neurons=1):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(neurons, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(4)))
	model.add(Dropout(0.2))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.723958 using {'neurons': 5}
0.703125 (0.016573) with: {'neurons': 1}
0.723958 (0.027126) with: {'neurons': 5}
0.697917 (0.001841) with: {'neurons': 10}
0.713542 (0.010253) with: {'neurons': 15}
0.718750 (0.026107) with: {'neurons': 20}
0.721354 (0.016053) with: {'neurons': 25}
0.717448 (0.020505) with: {'neurons': 30}
